# Test Sending File Descriptors
https://pythontic.com/modules/socket/send_fds

In [1]:
!pip install av

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 23.2 MB/s eta 0:00:0000:0100:01


In [ ]:
# Example Python program that creates 
# a server using an Unix Domain Socket  
# and receives File Descriptors from other
# processes connecting to this server through 
# the function socket.receive_fds()  
import socket
import os
from pathlib import Path

import av

socket_path = Path.home()/"test.sock"
socket_path.touch()

# Create a Unix Domain socket
domainServer = socket.socket(socket.AF_UNIX)

# Bind the socket to a path
os.unlink(str(socket_path))
domainServer.bind(str(socket_path))

# Enter into listening mode
domainServer.listen()  
print("Unix domain socket waiting for client connections...")

# Start accepting connections
while(True):
    sock, addr = domainServer.accept()

    # Read file descriptors from a connection
    while(True):
        tup = socket.recv_fds(sock, 1024, 2)
        type = tup[0]
        fds = tup[1][0]
        
        if type == b"url":
            request = b"GET / HTTP/1.1\r\nHost:www.example.com\r\n\r\n"
            os.write(fds, request)
            print(os.read(fds, 100))
        elif type == b"file":
            print(os.read(fds, 100))
        elif type == b"mp4":
            container = av.open(fds)
            for frame in container:
                print('decoded frame')

        sock.send("Great...have read all the news".encode())
        sock.close()
        break

Unix domain socket waiting for client connections...
b'HTTP/1.1 200 OK\r\nConnection: keep-alive\r\nContent-Length: 1256\r\nCache-Control: max-age=604800\r\nConten'
b'{"instruction":"How many torpedo tubes did the NameS class submarine have?","input":"","output":"6"}'
